In [ ]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [2]:
udahub_db = "data/core/udahub.db"

In [3]:
reset_db(udahub_db)

✅ Removed existing data/core/udahub.db
2025-08-07 01:47:21,701 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-07 01:47:21,701 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/core/udahub.db with fresh schema


In [4]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [5]:
account_id = "cultpass"
account_name = "CultPass Card"

In [6]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [7]:
# TODO: Create additional 10 articles

In [8]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

In [9]:
cultpass_articles

[{'title': 'How to Reserve a Spot for an Event',
  'content': 'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping \'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
  'tags': 'reservation, events, booking, attendance'},
 {'title': "What's Included in a CultPass Subscription",
  'content': 'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee (visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass s

In [ ]:
if len(cultpass_articles) < 14:
    raise AssertionError("You should load the articles with at least 14 records")

In [11]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb) 
    

**Ticket**

In [12]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [13]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [14]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])


# Tests

In [15]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

<Account(account_id='cultpass', account_name='CultPass Card')>


In [16]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    for article in account.knowledge_articles:
        print(article)

<Knowledge(article_id='21e466ef-11a9-4637-b970-859fcc89cff0', title='How to Reserve a Spot for an Event')>
<Knowledge(article_id='3ec5f050-9005-4b37-8b3a-1368fb882fd6', title='What's Included in a CultPass Subscription')>
<Knowledge(article_id='63a74f8d-41cf-4c86-b584-7ab99d19f090', title='How to Cancel or Pause a Subscription')>
<Knowledge(article_id='69102d73-4b2d-4b09-8e36-120840b0f64f', title='How to Handle Login Issues?')>


In [17]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

<User(user_id='a104aa35-5f37-4230-b247-21d24961e526', user_name='Alice Kingsley', external_user_id='a4ab87')>


In [18]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket:udahub.Ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)

<TicketMessage(message_id='a1f4aa78-fae6-4507-9fea-7a84b377061a', role='user', content='I can't log in to my Cultpass ...')>
